In [4]:
%load_ext autoreload

In [5]:
%autoreload 1
from qtim_ROP.learning.retina_net import RetiNet
from qtim_ROP.utils.image import imgs_by_class_to_th_array

In [21]:
import os
from os.path import join, basename, splitext
import cv2
import numpy as np

test_dir = "C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/images/First100/images"
prep_dir = "C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/images/First100/output_tf/preprocessed"
image_list = []
ground_truth = []
originals = []

for i, class_ in enumerate(['normal', 'pre-plus', 'plus']):
    
    for img in os.listdir(join(test_dir, class_)):
        
        img = cv2.imread(join(prep_dir, splitext(basename(img))[0] + '.bmp'))
        image_list.append(img)
        ground_truth.append(i)
        originals.append(img)
        
img_arr = np.stack(image_list, axis=0)
ground_truth = np.asarray(ground_truth)
originals = np.asarray(originals)

In [22]:
print(img_arr.shape)
import h5py

out_file = 'first100.h5'
with h5py.File(out_file, "w") as f:
    f.create_dataset('data', data=img_arr, dtype=img_arr.dtype)
    f.create_dataset('labels', data=ground_truth, dtype=ground_truth.dtype)
    f.create_dataset('original_files', data=originals, dtype=originals.dtype)


(100, 224, 224, 3)


In [23]:
import os
config = 'C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/models/python3_tf/classifiers/Split3_Classify_128features/Split3_Classify_128features.yaml'
h5_file = 'C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/images/splits/split_3/test.h5'

model = RetiNet(config)
model.model.load_weights(os.path.join(os.path.dirname(config), 'final_weights.h5'))

Evaluating model


In [24]:
from qtim_ROP.utils.image import create_generator 
val_gen, val_n, y_true, _ = create_generator(h5_file, model.model.input_shape[1:], training=False, batch_size=1, regression=False)

Loading data from HDF5 file 'C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/images/splits/split_3/test.h5'
Data was transposed to have channels last
Converted string labels to to integers
{'No': 0, 'Pre-Plus': 1, 'Plus': 2}


In [26]:
y_preda = model.model.predict_generator(val_gen)

In [25]:
from qtim_ROP.evaluation.metrics import plot_ROC_curves

plot_ROC_curves(y_true, y_preda, {0: 'Normal', 2: 'Plus'})

[{'method': 'ROC', 'label': 'Normal', 'score': 0.40533574952179596},
 {'method': 'ROC', 'label': 'Plus', 'score': 0.4053359683794466}]

In [ ]:
import h5py
import numpy as np
h5_file = 'C:/Users/jxb29/Dropbox (Partners HealthCare)/DeepROP_deployment/images/splits/split_3/test.h5'

test_data = h5py.File(h5_file, 'r')
data = test_data['data']
labels = list(test_data['labels'])
plus_idx = [i for i in range(len(labels)) if labels[i].decode("utf-8") == 'Pre-Plus'][:20]
plus_images = data[plus_idx, ...].transpose((0, 2, 3, 1))
print(plus_images.shape)

In [ ]:
y_pred = model.predict(plus_images)
print(y_pred)
print(np.argmax(y_pred, axis=1))